In [ ]:
import sys
import os
sys.path.append('/root/capsule/code/beh_ephys_analysis')
from utils.beh_functions import parseSessionID, session_dirs, get_unit_tbl, get_session_tbl
from utils.plot_utils import shiftedColorMap, template_reorder, get_gradient_colors
from utils.opto_utils import opto_metrics
from utils.ephys_functions import cross_corr_train, auto_corr_train, load_drift
import json
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from aind_ephys_utils import align
%matplotlib inline

In [ ]:
session = 'behavior_754897_2025-03-13_11-20-42'
data_type = 'curated'

In [ ]:
unit_tbl = get_unit_tbl(session, data_type)
opto_tbl = get_opto_tbl(session, data_type)